In [1]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import pandas as pd
import datetime
from catboost import CatBoostClassifier
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

import ast

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model

from tqdm import tqdm_notebook as tqdm
import gc, pickle

import datetime
from catboost import CatBoostClassifier
from time import time
from collections import Counter
from scipy import stats

from sklearn import preprocessing

In [2]:
def create_is_sell_data(sell_prices_df, calendar_df, train_df):
    sell_prices_df['id'] = sell_prices_df['item_id'].astype('str')+'_'+sell_prices_df['store_id']+'_validation'
    sell_prices_data = sell_prices_df[sell_prices_df.wm_yr_wk.isin(calendar_df.wm_yr_wk.unique())]
    sell_prices_data.reset_index(drop=True, inplace=True)
    tmp = sell_prices_data.groupby(['id'])[['wm_yr_wk', 'sell_price']].apply(
        lambda x: x.set_index('wm_yr_wk')['sell_price'].to_dict()
    ).to_dict()
    d = calendar_df.d
    wm_yr_wk = calendar_df.wm_yr_wk
    price_data = {}
    for col in tqdm(train_df.id.unique()):
        price_data[col] = wm_yr_wk.map(tmp[col])
    price_data = pd.DataFrame(price_data)
    price_data.index = d
    is_sell = price_data.notnull().astype(float).T
    price_data = price_data.fillna(0)
    
    is_sell.index=train_df.id
    train_df.index=train_df.id
    is_sell = pd.concat([
        train_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']], is_sell
    ], axis=1)
    price_data = pd.concat([
        train_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']], price_data.T  
    ], axis=1)
    
    return price_data, is_sell

def set_index(df, name):
    d = {}
    for col, value in df.iloc[0,:].items():
        if type(col)==str:
            if type(value)!=str:
                v = 'd'
            else:
                v='id'
        else:
            v=name
        d[col]=v
    return d

def dcol2int(col):
    if col[:2]=='d_':
        return int(col.replace('d_', ''))
    else:
        return col

In [32]:
%%time
#path = '/Users/kanoumotoharu/Downloads/m5-forecasting-accuracy/'
#path = '/Users/abcdm/Downloads/m5-forecasting-accuracy/'
path = '../input/m5-forecasting-accuracy/'

train_df = pd.read_csv(path+'sales_train_validation.csv')
calendar_df = pd.read_csv(path+'calendar.csv')
sell_prices_df = pd.read_csv(path+'sell_prices.csv')
sample_submission_df = pd.read_csv(path+'sample_submission.csv')

calendar_df['d'] = calendar_df.d.str.replace('d_', '').astype(int)
cols = train_df.columns
cols = [dcol2int(col) for col in cols]
train_df.columns=cols
calendar_df['date']=pd.to_datetime(calendar_df.date)
calendar_df.index = calendar_df.d
price_data, is_sell = create_is_sell_data(sell_prices_df, calendar_df, train_df)

d_cols = [ col for col in train_df.columns if type(col)!=str ]
for i in range(1,29):
    train_df[d_cols[-1]+i]=np.nan
d_cols = [ col for col in train_df.columns if type(col)!=str ]

train_df = pd.concat([
    train_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']],
    train_df[d_cols]+is_sell[d_cols].replace(0, np.nan).replace(1, 0)
], axis=1)


CPU times: user 1min 20s, sys: 5.71 s, total: 1min 25s
Wall time: 1min 24s


In [33]:
gc.collect()

0

In [38]:
train_d_cols = d_cols[-50:-28]
_state='CA'

train_df = train_df[train_df.state_id==_state];gc.collect()
ids = train_df.id.unique().tolist()
calendar_dict = calendar_df[[
    'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1',
    'event_type_1', 'event_name_2', 'event_type_2',f'snap_{_state}'
]].set_index('d').to_dict()
cat_dict = train_df[train_df.state_id==_state][['item_id',  'dept_id',   'cat_id', 'store_id', 'state_id']].to_dict()

sell_prices_df['key'] = sell_prices_df['wm_yr_wk'].astype(str)+'_'+sell_prices_df['id'].astype(str)
sell_prices_dict = sell_prices_df.set_index('key')['sell_price'].to_dict()

del calendar_df, sell_prices_df, is_sell; gc.collect()

train_df = train_df.loc[ids,d_cols].stack().reset_index()
train_df = train_df.rename(columns=set_index(train_df, 'TARGET'))
train_df.sort_values('d', inplace=True)
train_df.reset_index(drop=True, inplace=True)


for key, value in calendar_dict.items():
    train_df[key] = train_df['d'].map(value).fillna('NAN')
    
for key, value in cat_dict.items():
    train_df[key] = train_df['id'].map(value).fillna('NAN')
    
train_df['sell_price'] = (train_df.wm_yr_wk.astype(str)+'_'+train_df.id.astype(str)).map(sell_prices_dict)

SyntaxError: invalid syntax (<ipython-input-38-f02d29d79b49>, line 16)

In [31]:
train_df.groupby(['event_name_1', 'dept_id'])['TARGET'].transform(
    lambda x: x.sort_index().rolling(len(x), min_periods=1).mean()
)

0            0.000000
1           56.000000
2           30.000000
3            2.000000
4            1.000000
              ...    
18217793     2.818897
18217794     1.107529
18217795     1.587134
18217796     1.107529
18217797     2.818899
Name: TARGET, Length: 18217798, dtype: float64

In [9]:
data = train_df[train_df.d.isin(train_d_cols)]#.stack(dropna=False).reset_index()
data.reset_index(inplace=True, drop=True)

,id,d,TARGET,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,dept_id,cat_id,store_id,state_id
0,FOODS_1_012_CA_2_validation,1892,17.0,2016-04-03,11610,Sunday,2,4,2016,NaN,NaN,NaN,NaN,1,FOODS_1_012,FOODS_1,FOODS,CA_2,CA
1,HOUSEHOLD_2_191_CA_1_validation,1892,0.0,2016-04-03,11610,Sunday,2,4,2016,NaN,NaN,NaN,NaN,1,HOUSEHOLD_2_191,HOUSEHOLD_2,HOUSEHOLD,CA_1,CA
2,FOODS_3_777_CA_1_validation,1892,2.0,2016-04-03,11610,Sunday,2,4,2016,NaN,NaN,NaN,NaN,1,FOODS_3_777,FOODS_3,FOODS,CA_1,CA
3,HOUSEHOLD_2_487_CA_1_validation,1892,0.0,2016-04-03,11610,Sunday,2,4,2016,NaN,NaN,NaN,NaN,1,HOUSEHOLD_2_487,HOUSEHOLD_2,HOUSEHOLD,CA_1,CA
4,HOBBIES_1_381_CA_2_validation,1892,5.0,2016-04-03,11610,Sunday,2,4,2016,NaN,NaN,NaN,NaN,1,HOBBIES_1_381,HOBBIES_1,HOBBIES,CA_2,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268307,FOODS_3_201_CA_3_validation,1913,5.0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,FOODS_3_201,FOODS_3,FOODS,CA_3,CA
268308,HOBBIES_1_354_CA_3_validation,1913,15.0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,HOBBIES_1_354,HOBBIES_1,HOBBIES,CA_3,CA
268309,HOUSEHOLD_1_257_CA_2_validation,1913,2.0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,HOUSEHOLD_1_257,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA
268310,HOBBIES_1_269_CA_3_validation,1913,0.0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,HOBBIES_1_269,HOBBIES_1,HOBBIES,CA_3,CA


In [ ]:
train_df.groupby(['event_name_1', 'dept_id'])['TARGET'].transform(
    lambda x: x.rolling(1).mean()
)

In [ ]:
def c_diff_data(data, _diff, wins):
    data['tmp'] = data.groupby(['id'])['TARGET'].transform(
        lambda x: x.diff(_diff)
    )
    
    for win in wins:
        data[f'diff{_diff}_{shift}'] = data.groupby(['id'])['tmp'].transform(
            lambda x: x.rolling(win).mean()
        )
        data[f'abs_diff{_diff}_{shift}'] = data.groupby(['id'])['tmp'].transform(
            lambda x: abs(x).rolling(win).mean()
        )
    return data

In [ ]:
def c_snape_data(data, calendar_df, _state):
    for shift in [-3,-2,1,0,1,2,3]:
        if shift<0:
            data[f'mns{abs(shift)}_snap{_state}'] = data.d.map(calendar_df.set_index('d')[f'snap_{_state}'].shift(shift).to_dict())
        else:
            data[f'{abs(shift)}_snap{_state}'] = data.d.map(calendar_df.set_index('d')[f'snap_{_state}'].shift(shift).to_dict())
    return data

In [ ]:
def tmp_func(x):
    print(x)
    retu

In [ ]:
data.groupby(['event_name_1'])['TARGET'].mean()#.transform(
  #  lambda x: x.rolling(10).mean()
#)